In [1]:
import yfinance as yf
import pandas as pd

In [15]:
stock_df = yf.download('TCS.NS', start='2015-01-01', end='2025-01-01')
stock_df.columns = stock_df.columns.get_level_values(0)
stock_df.reset_index(inplace=True)
stock_df.head()

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
0,2015-01-01,1058.276001,1067.193511,1056.384383,1067.193511,366830
1,2015-01-02,1072.369629,1077.150590,1060.375713,1060.541966,925740
2,2015-01-05,1056.072510,1080.871101,1049.586993,1073.013738,1754242
3,2015-01-06,1017.139282,1051.437487,1014.395385,1051.437487,2423784
4,2015-01-07,1005.124329,1030.671271,1000.863037,1026.867329,2636332


In [2]:
nifty_df = yf.download("^NSEI", start='2015-01-01', end='2025-01-01')  # Nifty50 index
nifty_df.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
Date,,,,,
2015-01-02,8395.450195,8410.599609,8288.700195,8288.700195,101900
2015-01-05,8378.400391,8445.599609,8363.900391,8407.950195,118200
2015-01-06,8127.350098,8327.849609,8111.350098,8325.299805,172800
2015-01-07,8102.100098,8151.200195,8065.450195,8118.649902,164100
2015-01-08,8234.599609,8243.500000,8167.299805,8191.399902,143800


In [6]:
nifty_df.rename(columns={"Close": "Nifty_Close"}, inplace=True)
nifty_df.columns = nifty_df.columns.get_level_values(0)
nifty_df.head()

Price,Nifty_Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,8395.450195,8410.599609,8288.700195,8288.700195,101900
2015-01-05,8378.400391,8445.599609,8363.900391,8407.950195,118200
2015-01-06,8127.350098,8327.849609,8111.350098,8325.299805,172800
2015-01-07,8102.100098,8151.200195,8065.450195,8118.649902,164100
2015-01-08,8234.599609,8243.500000,8167.299805,8191.399902,143800


In [9]:
nifty_df = nifty_df[['Nifty_Close']]
nifty_df.head()

Price,Nifty_Close
Date,
2015-01-02,8395.450195
2015-01-05,8378.400391
2015-01-06,8127.350098
2015-01-07,8102.100098
2015-01-08,8234.599609


In [8]:
nifty_df.columns

Index(['Nifty_Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

In [12]:
df = pd.merge(stock_df, nifty_df[['Nifty_Close']], on="Date", how="left")
df.head()

Price,Close,High,Low,Open,Volume,Nifty_Close
Date,,,,,,
2015-01-01,1058.276001,1067.193511,1056.384383,1067.193511,366830,NaN
2015-01-02,1072.369629,1077.150590,1060.375713,1060.541966,925740,8395.450195
2015-01-05,1056.072510,1080.871101,1049.586993,1073.013738,1754242,8378.400391
2015-01-06,1017.139282,1051.437487,1014.395385,1051.437487,2423784,8127.350098
2015-01-07,1005.124329,1030.671271,1000.863037,1026.867329,2636332,8102.100098


In [14]:
df['Nifty_Returns'] = df['Nifty_Close'].pct_change(fill_method=None)
df['Nifty_Lag_1'] = df['Nifty_Close'].shift(1)
df['Nifty_MA10'] = df['Nifty_Close'].rolling(window=10).mean()

df.head()

Price,Close,High,Low,Open,Volume,Nifty_Close,Nifty_Returns,Nifty_Lag_1,Nifty_MA10
Date,,,,,,,,,
2015-01-01,1058.276001,1067.193511,1056.384383,1067.193511,366830,NaN,NaN,NaN,NaN
2015-01-02,1072.369629,1077.150590,1060.375713,1060.541966,925740,8395.450195,NaN,NaN,NaN
2015-01-05,1056.072510,1080.871101,1049.586993,1073.013738,1754242,8378.400391,-0.002031,8395.450195,NaN
2015-01-06,1017.139282,1051.437487,1014.395385,1051.437487,2423784,8127.350098,-0.029964,8378.400391,NaN
2015-01-07,1005.124329,1030.671271,1000.863037,1026.867329,2636332,8102.100098,-0.003107,8127.350098,NaN


In [16]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from src.feature_engineering import add_features


def forecast_with_lstm(stock_name: str, forecast_days: int = 7, lookback: int = 30):
    try:
        # Load processed data
        data_path = f"C://GITHUB CODES//stock-predictor-ml//data/processed/{stock_name}_v2.pkl"
        if not os.path.exists(data_path):
            print(f"❌ Processed file not found: {data_path}")
            return

        df = pd.read_pickle(data_path)
        df = add_features(df)
        df.dropna(inplace=True)

        # Use only closing price for LSTM
        data = df[['Close']].values

        # Normalize
        scaler = MinMaxScaler(feature_range=(0, 1))
        data_scaled = scaler.fit_transform(data)

        # Create sequences
        X, y = [], []
        for i in range(lookback, len(data_scaled)):
            X.append(data_scaled[i - lookback:i, 0])
            y.append(data_scaled[i, 0])

        X, y = np.array(X), np.array(y)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build LSTM model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X, y, epochs=20, batch_size=32, verbose=1)

        # Forecast
        last_sequence = data_scaled[-lookback:]
        forecasted = []

        for _ in range(forecast_days):
            input_seq = last_sequence[-lookback:].reshape(1, lookback, 1)
            pred_scaled = model.predict(input_seq, verbose=0)[0, 0]
            forecasted.append(pred_scaled)
            last_sequence = np.append(last_sequence, pred_scaled)[-lookback:]

        forecasted_prices = scaler.inverse_transform(np.array(forecasted).reshape(-1, 1)).flatten()
        future_dates = pd.date_range(start=df['Date'].max() + pd.Timedelta(days=1), periods=forecast_days)

        forecast_df = pd.DataFrame({
            'Date': future_dates,
            'LSTM_Forecast': forecasted_prices
        })

        os.makedirs("reports", exist_ok=True)
        forecast_df.to_csv(f"C://GITHUB CODES//stock-predictor-ml//reports/{stock_name}_lstm_forecast.csv", index=False)
        print(f"\n✅ LSTM forecast saved to reports/{stock_name}_lstm_forecast.csv")

    except Exception as e:
        print(f"❌ Error during LSTM forecast: {str(e)}")


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'